In [1]:
# 파이썬 버전 확인
!python --version

Python 3.8.5


In [3]:
!pip install --upgrade pip

In [6]:
!python --version

!pip install --upgrade pip

!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

!pip install konlpy

Python 3.8.5
Processing c:\users\smhrd\머신러닝\jpype1-1.1.2-cp38-cp38-win_amd64.whl
JPype1 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [7]:
from konlpy.tag import Kkma

kkma = Kkma()

In [10]:
import pandas as pd

In [14]:
df_train = pd.read_csv('nsmc-master/ratings_train.txt', delimiter='\t')

In [15]:
df_train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [18]:
# 결측치 제거
df_train.dropna(inplace = True)

In [19]:
df_test = pd.read_csv('nsmc-master/ratings_test.txt', delimiter='\t')

In [20]:
df_test.dropna(inplace = True)

In [21]:
X_train = df_train['document']
y_train = df_train['label']
X_test = df_test['document']
y_test = df_test['label']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [51]:
X_train[:3]
tfidf.fit(X_train[:3])

TfidfVectorizer()

In [52]:
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [35]:
kkma =Kkma()

In [ ]:
# 문장단위 토큰화 : sentences()
# 명사단위 토큰화 : nouns()
# 형태소단위 토큰화 : morphs()

In [36]:
X_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [37]:
kkma.morphs(X_train[0])

['아', '아', '더빙', '..', '진짜', '짜증나', '네요', '목소리']

In [39]:
kkma.nouns(X_train[0])

['더빙', '목소리']

In [53]:
kkma.pos(X_train[0])

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('..', 'SW'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [57]:
d = pd.DataFrame(kkma.pos(X_train[0]),columns=['mprhs','tag'])

In [59]:
d.set_index('tag',inplace=True)

In [60]:
d.loc[['VV','NNG']]

,mprhs
tag,
VV,아
VV,짜증나
NNG,더빙
NNG,목소리


In [61]:
# tfidf의 토큰화 방법 변경

In [63]:
# 명사 단위로 토큰화
def myTokenizer(text):
    return kkma.nouns(text)

In [64]:
# tfidf에 내가 만든 토큰 방법 적용
tfidf = TfidfVectorizer(tokenizer = myTokenizer)

In [66]:
tfidf.fit(X_train[:3])
tfidf.vocabulary_

{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

In [67]:
# VV, VA, NNG의 형태소만 토큰화
def myTokenizer2(text):
    d = pd.DataFrame(kkma.pos(text), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index):
        labels = ['VV','VA','NNG']
        return d.loc[d.index.intersection(labels)].dropna()['morph'].values
    else :
        return []

In [68]:
tfidf = TfidfVectorizer(tokenizer = myTokenizer2)
tfidf.fit(X_train[:3])
tfidf.vocabulary_

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [69]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [70]:
pipe = make_pipeline(TfidfVectorizer(tokenizer = myTokenizer2), LinearSVC())

In [71]:
pipe.fit(X_train[:100],y_train[:100])

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x000001A2AD05B160>)),
                ('linearsvc', LinearSVC())])

In [90]:
pipe.steps[0][1]

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x000001A2AD05B160>)

In [76]:
# tifdf
pipe_tfidf = pipe.steps[0][1]
#svm
pipe_svm = pipe.steps[1][1]

In [72]:
pipe.steps[0][1].vocabulary_

{'아': 281,
 '짜증나': 431,
 '더빙': 114,
 '목소리': 175,
 '흠': 497,
 '포스터': 473,
 '보고': 205,
 '영화': 315,
 '줄': 418,
 '오버': 320,
 '연기': 306,
 '가볍': 4,
 '재': 389,
 '추천': 444,
 '어': 297,
 '보': 204,
 '교도소': 37,
 '이야기': 357,
 '재미': 390,
 '점': 400,
 '조정': 407,
 '멀': 165,
 '없': 303,
 '익살': 364,
 '스파이': 265,
 '더': 113,
 '맨': 162,
 '커스틴': 451,
 '돋보이': 118,
 '늙': 100,
 '보이': 209,
 '하': 478,
 '이쁘': 356,
 '걸음마': 23,
 '초등학교': 440,
 '학년': 480,
 '생인': 242,
 '용': 328,
 '별': 203,
 '반개': 190,
 '떼': 135,
 '아깝': 282,
 '원작': 339,
 '긴장감': 53,
 '살리': 234,
 '욕': 326,
 '이응': 358,
 '경': 27,
 '길': 54,
 '우': 330,
 '생활': 243,
 '발': 193,
 '납': 76,
 '감금': 10,
 '반복': 191,
 '이': 351,
 '드라마': 126,
 '가족': 7,
 '사람': 229,
 '모': 172,
 '엿': 313,
 '나오': 71,
 '낫': 78,
 '액션': 294,
 '안': 288,
 '있': 376,
 '평점': 470,
 '우드': 331,
 '식': 271,
 '화려': 493,
 '낮': 80,
 '헐': 488,
 '길들이': 55,
 '인피': 367,
 '니트': 103,
 '짱': 432,
 '나': 66,
 '죽': 417,
 '때': 133,
 '눈물': 95,
 '향수': 486,
 '자극': 380,
 '허': 487,
 '진호': 427,
 '감성': 13,
 '절제': 399,
 '멜로': 1

In [79]:
voc = pd.DataFrame(pipe_tfidf.vocabulary_.keys(),
            index = pipe_tfidf.vocabulary_.values())

In [80]:
voc.sort_index()

,0
0,ㄱ
1,ㅇ
2,ㅈ
3,가
4,가볍
...,...
495,활력
496,횡단보도
497,흠
498,흥행


In [81]:
# Series로 확인
voc.sort_index()[0]

0         ㄱ
1         ㅇ
2         ㅈ
3         가
4        가볍
       ... 
495      활력
496    횡단보도
497       흠
498      흥행
499       희
Name: 0, Length: 500, dtype: object

In [83]:
pipe_svm.coef_.T

array([[ 4.36018797e-01],
       [-6.03072371e-01],
       [-3.75531339e-01],
       [-6.69214653e-02],
       [ 7.64266079e-02],
       [ 2.71460015e-01],
       [ 4.73610925e-01],
       [-1.57267587e-01],
       [-1.04484369e-01],
       [-4.04421939e-01],
       [-1.57267587e-01],
       [-6.93464285e-02],
       [-2.30526308e-01],
       [ 2.04477847e-01],
       [ 4.73610925e-01],
       [ 3.65072242e-01],
       [ 1.61070890e-01],
       [-2.26962049e-01],
       [-2.01024124e-01],
       [ 1.25193284e-01],
       [ 3.28339488e-01],
       [-2.30374303e-01],
       [ 7.06696802e-02],
       [-2.67125813e-01],
       [ 2.33208210e-01],
       [-2.72703798e-01],
       [-3.68656579e-01],
       [-1.57267587e-01],
       [-1.34807313e-01],
       [-3.02067698e-01],
       [ 2.70640909e-01],
       [-3.49120336e-01],
       [ 5.63366953e-01],
       [-2.75557025e-01],
       [-2.58282819e-01],
       [ 2.60283096e-01],
       [-8.79503484e-01],
       [-3.53319710e-01],
       [-1.4

In [87]:
# 토큰들과 가중치를 연결해서 각각의 토큰들이 긍정 / 부정에 얼마나 영향을 끼쳤는지 확인
result = pd.DataFrame(pipe_svm.coef_.T, index = voc.sort_index()[0],
                      columns = ['w'])

In [89]:
result.sort_values(by = "w", ascending = False)

,w
0,
재미있,0.928474
재밌,0.895684
인상적,0.761267
드라마,0.725359
내일,0.661579
...,...
추천,-0.673392
실망,-0.675714
졸작,-0.675715
